In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

## The below two code cells:

- Takes a list of Central Index Keys ("CIK"), which are numeric identifiers that identify a particular SEC filing entity.


- Uses the CIK to construct a  URL that identifies a company's "13F-HR" filings, which contain the data of interest.
     
     - The first part of the URL, "url_first" has the root of the URL and expects a CIK
     - The CIK identifies the company
     - url_second specifies the "type" of filing as "13F-HR' and supplies other necessary parameters, such as a count of the number of rows to show.
     

## Some additional notes:
- I originally had a three-part url designed to pull more filings, but it caused errors and ended up being unnecessary since all of the filings of interest for each company appeared on the first page.  This is because prior to 2013, the filings were made in plain .txt files.

    - If I wanted to expand upon this project, I would need to pull in _all_ filings, which would require cutomizing a function to read the text filings as well as some error checking on the first cell below that pulls in all of the "doclink" values.
    
    
- I imported the time module and added `time.sleep()` calls to force the requesting part of the loop to wait between requests so as not to spam the SEC.

In [2]:
### DON'T MESS WITH THIS CELL -- IT IS FINE
url_first = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='
url_second = '&type=13F-HR%25&dateb=&owner=exclude&start=0&count=40'
CIK = ['0001167483']
doclink = []

df = pd.DataFrame(columns=['manager', 'filing_date', 'report_date', 'sh_name', 'sh_class', 'cusip', 'val1000', 'share_count', 
                                 'share_or_prin', 'discretion','sole_vote_amt', 'shared_vote_amt'])

pos_df= pd.DataFrame(columns=['manager', 'filing_date', 'report_date', 'sh_name', 'sh_class', 'cusip', 'val1000', 'share_count', 
                                 'share_or_prin', 'discretion','sole_vote_amt', 'shared_vote_amt'])
sh_name = []
sh_class = []
cusip = []
val1000 = []
share_count = []
share_or_prin = []
discretion = []
sole_vote_amt = []
shared_vote_amt = []
mgr = []
filing_date = []
report_date = []

        

                

### verify=False only on  in the "requests" calls below for when I run this at PDT.  Sean Ramos couldn't figure out how to get Anaconda to play nicely with our certs.

## I'm pretty sure the cell below does what I want it to, which is go through and grab the documents link for every filing labled "13F-HR".

- I'm aware that my "for row" loop is probably very inelegant and inefficient.

## I viewed `sorted(doclink)` and it appears to be working, but my error _could_ be in this cell too, if I'm grabbing the same doclink more than once.

In [5]:
CIK = ['0001167483']
for cik in CIK:
    url = url_first+cik+url_second
    time.sleep(.33)
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    mgr_name = str.strip(soup.find('span', {'class': 'companyName'}).text.split('CIK')[0])
    file_table = soup.find('table', {'class':'tableFile2'})
    rows = file_table.find_all('tr')

    for row in rows:
        cells = row.find_all('td')
        for cell in cells[0:1]:
            if cells[0].text == '13F-HR':
                doclink.append(cells[1].find(href=True).get('href'))
                    

C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
doclink

['/Archives/edgar/data/1167483/000091957418005626/0000919574-18-005626-index.htm',
 '/Archives/edgar/data/1167483/000091957418003754/0000919574-18-003754-index.htm',
 '/Archives/edgar/data/1167483/000091957418001706/0000919574-18-001706-index.htm',
 '/Archives/edgar/data/1167483/000091957417008069/0000919574-17-008069-index.htm',
 '/Archives/edgar/data/1167483/000091957417006218/0000919574-17-006218-index.htm',
 '/Archives/edgar/data/1167483/000091957417004286/0000919574-17-004286-index.htm',
 '/Archives/edgar/data/1167483/000091957417001974/0000919574-17-001974-index.htm',
 '/Archives/edgar/data/1167483/000091957416016596/0000919574-16-016596-index.htm',
 '/Archives/edgar/data/1167483/000091957416014985/0000919574-16-014985-index.htm',
 '/Archives/edgar/data/1167483/000091957416013223/0000919574-16-013223-index.htm',
 '/Archives/edgar/data/1167483/000091957416011040/0000919574-16-011040-index.htm',
 '/Archives/edgar/data/1167483/000091957415008225/0000919574-15-008225-index.htm',
 '/A

## This cell below is supposed to go through every "doclink" grabbed in the cell above and do the following:

- Populate the filing date in a list ((i.e., if its a Q1 filing, "05-15-2018")
- Populate the "period of report" (i.e., if its a Q1 filing, "03-30-2018")
- Populate the name of the company making the filing.
- Grab all of the "infotable.xml" links, parse them, and store the data in lists

## I'm pretty sure that my error is in the "for cell in cells" loop.  In all past scraping work I've done I've been able to pull back a single html tag by using an attribute of the given tag type.  Here, I don't really have a good way to do that.  I'm pretty sure that this code is doing bullet 3 above multiple times per "doclink"

In [6]:
# info_table_link = []  Maybe store the links here and loop through them along with the below?
# primary_doc_link = [] Maybe store the xml link to the "primary doc xml" file and use it pull manager, filing date, and 
# period of report?

for link in doclink:
    url = 'https://www.sec.gov' + link
    time.sleep(.33)
    response = requests.get(url, verify=False)
    filing_soup = BeautifulSoup(response.text, 'lxml')

    mgr_name = str.strip(filing_soup.find('span', {'class': 'companyName'}).text.split('(')[0])
    
    form_grouping = filing_soup.find_all('div', {'class': 'info'})
    report_dt = form_grouping[3].text
    filing_dt = form_grouping[4].text

    cells = filing_soup.find_all('td')
    for cell in cells:

#             if cell.text == 'primary_doc.xml':
#                 response = requests.get('https://www.sec.gov' + cell.find(href=True).get('href'),verify=False)
#                 fsoup = BeautifulSoup(response.text, "lxml")
#                 report_dt= fsoup.signaturedate.text
#                 filing_dt= fsoup.periodofreport.text

        if (cell.text == 'form13fInfoTable.xml') | (cell.text == 'infotable.xml'):
            response = requests.get('https://www.sec.gov' + cell.find(href=True).get('href'), verify=False)
            soup = BeautifulSoup(response.text, "lxml")
            record = soup.find_all('infotable')
            for rec in record:
                mgr.append(mgr_name)
                report_date.append(report_dt)
                filing_date.append(filing_dt)
                sh_name.append(rec.find('nameofissuer').text)
                sh_class.append(rec.find('titleofclass').text)
                cusip.append(rec.find('cusip').text)
                val1000.append(rec.find('value').text)
                share_count.append(rec.find('sshprnamt').text)
                share_or_prin.append(rec.find('sshprnamttype').text)
                discretion.append(rec.find('investmentdiscretion').text)
                sole_vote_amt.append(rec.find('sole').text)
                shared_vote_amt.append(rec.find('shared').text)

C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxa

C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxa

C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxa

C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxa

C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxabend\AppData\Local\conda\conda\envs\maxabend\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\maxa

In [7]:
df['manager'] = mgr
df['filing_date'] = filing_date
df['report_date']= report_date
df['sh_name'] = sh_name
df['sh_class'] = sh_class
df['share_count'] = share_count
df['cusip'] = cusip
df['val1000'] = val1000
df['share_or_prin'] = share_or_prin
df['discretion'] = discretion
df['sole_vote_amt'] = sole_vote_amt
df['shared_vote_amt'] = shared_vote_amt


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880 entries, 0 to 1879
Data columns (total 12 columns):
manager            1880 non-null object
filing_date        1880 non-null object
report_date        1880 non-null object
sh_name            1880 non-null object
sh_class           1880 non-null object
cusip              1880 non-null object
val1000            1880 non-null object
share_count        1880 non-null object
share_or_prin      1880 non-null object
discretion         1880 non-null object
sole_vote_amt      1880 non-null object
shared_vote_amt    1880 non-null object
dtypes: object(12)
memory usage: 176.3+ KB


In [ ]:
df.to_csv("Comfirm.csv")

In [ ]:
mgr

# This is a cell which shouldn't be needed, but just in case.

### The idea here is pulling in filing date via an alternate method.

In [ ]:
url_first = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='
CIK = ['0001564702', '0001067983']
url_second = '&type=13F-HR%25&dateb=&owner=exclude&start='
url_third = '&count=100'
doclink = []
file=[]

for cik in CIK:
    mgr_name = str.strip(soup.find('span', {'class': 'companyName'}).text.split('CIK')[0])
    for num in range(0,300,100):
        url = url_first+cik+url_second+str(num)+url_third
        time.sleep(1)
        response = requests.get(url, verify=False)
        file_table = BeautifulSoup(response.text, 'lxml').find('table', {'class':'tableFile2'})
        rows = file_table.find_all('tr')
        
        for row in rows:
            cells = row.find_all('td')
            for cell in cells[0:1]:
                if cells[0].text == '13F-HR':
                    doclink.append(cells[1].find(href=True).get('href'))
            for cell in cells:
                file.append(cells[3].text)

    file = [x for i, x in enumerate(file) if i%5 == 0]

In [ ]:
file_table = soup.find('table', {'class':'tableFile2'})
rows = file_table.find_all('tr')
link = []
for row in rows:
    cells = row.find_all('td')
    for cell in cells[0:1]:
        if cells[0].text == '13F-HR':
            link.append(cells[1].find(href=True).get('href'))